# Equity Linked Notes - Reverse Convertible (single)

This notebook demonstrates how to access and use the functionalities of **Reverse Convertible Structured Notes** which are part of our **QPS** module within LSEG Financial Analytics SDK. This notebook presents a Reverse Convertible structure with single underlying.

**You will be able to:**
* Define Reverse Convertible instrument with basic parameters
* Configure pricing parameters
* Evaluate Reverse Convertible Analytics

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.structured_products` - for Structured Products instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import structured_products as sp 

import json
import datetime as dt
import pandas as pd
from IPython.display import display

## Data Preparation

To define a Structured Product instrument, in this example Reverse Convertible, you need to follow a structured 3-step process:
* **Structured Product Definition** - Specify basic Structured Product parameters (strike, dates, notional, index)
* **Structured Product Instrument Definition** - Create the instrument object
* **Pricing Preferences** - Configure pricing parameters, optional

In [2]:
# 1. Create SP definition object

SN_definition = sp.StructuredProductsDefinition(
    deal_ccy = "EUR",
    instrument_tag = "Reverse_Convertible",
    inputs = [
        sp.NameTypeValue(name="notional", type = "string", value="1000"),
        sp.NameTypeValue(name="Underlying", type = "string", value="ASML"),
        sp.NameTypeValue(name="StrikeDate", type = "date", value="14/08/2025"),
        sp.NameTypeValue(name="MaturityDate", type = "date", value="21/08/2030"),
        sp.NameTypeValue(name="LastValuationDate", type = "date", value="14/08/2030"),
        sp.NameTypeValue(name="Schedule", type = "schedule", value=[
                    ["14/08/2026", "14/08/2026", "21/08/2026", "21/08/2026"],
                    ["16/08/2027", "16/08/2027", "23/08/2027", "23/08/2027"],
                    ["14/08/2028", "14/08/2028", "21/08/2028", "21/08/2028"],
                    ["14/08/2029", "14/08/2029", "21/08/2029", "21/08/2029"]]),
        sp.NameTypeValue(name="RedemptionBarrier", type = "string", value="30%"),
        sp.NameTypeValue(name="CouponRate", type = "string", value="1.75%"),
        sp.NameTypeValue(name="Leverage", type = "string", value="100%")
    ],
    payoff_description = [
          [
            "Schedule type",
            "Schedule description",
            "Index",
            "Performance",
            "Coupon",
            "Settlement",
            "OptionAtMaturity",
            "PriceIn%",
            "Price"
          ],
          [
            "AtDate",
            "StrikeDate",
            "EqSpot(Underlying)",
            "",
            "",
            "",
            "",
            "",
            ""
          ],
          [
            "OnUserSchedule",
            "Schedule",
            "EqSpot(Underlying)",
            "",
            "Receive CouponRate",
            "",
            "",
            "",
            ""
          ],
          [
            "AtDate",
            "LastValuationDate",
            "EqSpot(Underlying)",
            "Index[t] / Index[StrikeDate]",
            "Receive (MaturityDate, CouponRate)",
            "Receive (MaturityDate, 100%)",
            "Receive (MaturityDate, If(Performance[t]>=RedemptionBarrier,0,(Performance[t] / Leverage -1)))",
            "Report((columnval(Coupon)+columnval(OptionAtMaturity)+columnval(Settlement))*100)",
            "Report(columnval(PriceIn%)/100*Notional)"
          ]
        ]
)


# 2. Create SP instrument definition object

rev_convert_single = sp.StructuredProductsDefinitionInstrument(definition = SN_definition)
print("Instrument definition created")


# 3. Create SP parameters object - optional

rev_convert_single_pricing_params = sp.StructuredProductsPricingParameters(
    valuation_date= dt.date(2025, 9, 10),  # Set your desired valuation date
    report_ccy="EUR",  # Set your reporting currency
    numerical_method = sp.GenericNumericalMethod(method="MonteCarlo"),
    models=[sp.ModelDefinition(
                  underlying_code = "ASML.AS",
                  underlying_name = "ASML",
                  underlying_currency = "EUR",
                  asset_class = "Equity",
                  model_name= "Dupire")
          ]
)

print("Pricing parameters configured")

Instrument definition created
Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing

try:
    response = sp.price(
        definitions=[rev_convert_single],
        pricing_preferences=rev_convert_single_pricing_params,
        market_data=None,
        return_market_data=True,  # or False
        fields=None  # or specify fields as a string
    )

    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing execution completed")
    
except Exception as e:
    print(f"Price calculation failed: {str(e)}")
    raise

Pricing execution completed


## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., strike, dates, notional, underlying, barrier, profit target), it's StructuredProductDefinition that we used. 

 - **pricingPreferences**: Valuation date, financial model, numerical method.

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashflows**: Includes arrays and detailed `cashFlows` (payments, fixings)
   - **description**: Instrument summary StructuredProductDefinition and also the default fields not specified in the StructuredProductDefinition, but used by default in the calculation
   - **greeks**: Sensitivities like `deltaAmountInDealCcy`, `gammaAmountInDealCcy`, `thetaAmountInDealCcy`, `vegaAmountInDealCcy`
   - **pricingAnalysis**: `valuationDate`, `marketDataDate`
   - **valuation**: `marketValueInDealCcy`
   - **error**: Empty if no issues

#### Description
Useful for understanding which fields are included by default in the price function, even if they are not explicitly specified.

In [4]:
# Extract description from response
description = response.data.analytics[0].description

# Convert to dictionary for display
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentTag": "Reverse_Convertible",
    "dealCcy": "EUR",
    "reportCcy": "EUR",
    "discountCurveId": "IRCurve_EUREURIBORSwapZCCurve_0001-01-01T00:00:00",
    "discountCurveName": "EUR EURIBOR Swap ZC Curve",
    "outputList": {
        "Coupon": 0.082364622,
        "Index": 3268.387890253,
        "OptionAtMaturity": -0.110201855,
        "Performance": 0.979744359,
        "Price": 870.548250853,
        "PriceIn%": 87.054825085,
        "Settlement": 0.898385484
    },
    "processingInformation": "'PricePercent' column was not retrieved in payoff description: related fields will not be retrieved. The following greeks could not be retrieved: Theta, Vega, Gamma, Delta. Make sure a 'PricePercent' column is specified in payoff description to compute greeks. "
}


#### Analytics

##### Valuation

In [5]:
# Extract vauation from the response
valuation = response.data.analytics[0].valuation

# Convert the dictionary to a DataFrame
df_rev_convert_single_valuation = pd.DataFrame(list(valuation.items()), columns=["Field", "Value"])

display(df_rev_convert_single_valuation)

,Field,Value
0,marketValueInDealCcy,870.548251
1,marketValueInReportCcy,870.548251


##### Cash Flows

In [6]:
# Extract cashflows from response
cashflows = response.data.analytics[0].cashflows["cashFlows"]

# Extract underlyings
model_df = pd.DataFrame(data=rev_convert_single_pricing_params.models)
underlying_list = model_df['underlyingName'].to_list()

# Build dataframes for all cash flow types
output = []
for cf_type in cashflows:
    cashflow_df = pd.DataFrame(cf_type['payments'])
    if cf_type['legTag'] == 'Index':
        cashflow_df = cashflow_df.rename(columns={'amount': underlying_list[0]})
    else:
        cashflow_df = cashflow_df.rename(columns={'amount': cf_type['legTag']})
    cashflow_df['discountFactor'] = cashflow_df['discountFactor'].round(4)
    output.append(cashflow_df)

# Merge all dataframes on the 'date' column
combined_df = pd.concat([*output],axis=1)

# Remove duplicated columns with the same values
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

common_cols = ['date', 'discountFactor', 'Performance', 'Coupon', 'Settlement', 'OptionAtMaturity', 'Price', 'currency', 'occurence']
indv_perf = [f'PERF_{underlying}' for underlying in underlying_list if len(underlying_list) > 1]
cols_to_display = common_cols[0:2] + underlying_list + indv_perf + common_cols[2:]

# Leave only columns to display
combined_df = combined_df.loc[:,[*cols_to_display]]

# Display the combined dataframe
display(combined_df)

,date,discountFactor,ASML,Performance,Coupon,Settlement,OptionAtMaturity,Price,currency,occurence
0,2025-08-14,1.0000,642.900000,0.000000,0.000000,0.000000,0.000000,0.000000,EUR,Historical
1,2026-08-14,0.9821,686.552299,0.000000,0.017494,0.000000,0.000000,0.000000,EUR,Future
2,2027-08-16,0.9633,692.858265,0.000000,0.017493,0.000000,0.000000,0.000000,EUR,Future
3,2028-08-14,0.9430,693.603747,0.000000,0.017493,0.000000,0.000000,0.000000,EUR,Future
4,2029-08-14,0.9213,697.640521,0.000000,0.017492,0.000000,0.000000,0.000000,EUR,Future
5,2030-08-14,0.8988,700.777440,1.090026,0.017491,0.999509,-0.122606,968.538217,EUR,Future


##### Greeks

In [7]:
# Extract Greeks from the response
greeks = response.data.analytics[0].greeks

# Convert the dictionary to a DataFrame
df_df_rev_convert_single_greeks = pd.DataFrame(list(greeks.items()), columns=["Greeks", "Value"])

display(df_df_rev_convert_single_greeks)

,Greeks,Value
0,deltaAmountInDealCcy,3.853830
1,deltaAmountInReportCcy,3.853830
2,gammaAmountInDealCcy,6.084050
3,gammaAmountInReportCcy,6.084050
4,vegaAmountInDealCcy,-9.388590
5,vegaAmountInReportCcy,-9.388590
6,thetaAmountInDealCcy,0.104725
7,thetaAmountInReportCcy,0.104725
